In [0]:
import os
import json
from openai import AzureOpenAI
from pyspark.sql.functions import *
from IPython.display import Markdown, display
from dotenv import load_dotenv

In [0]:
load_dotenv(r"C:\Users\HP\OneDrive\Documents\credentials.env")
api_key = os.environ['API_KEY']
api_base = os.environ['RESOURCE_ENDPOINT']
chat_model_id=os.environ['chat_model_id']
chat_model=os.environ['chat_model']
api_type = "azure"
api_version = "2023-06-01-preview"

openai = AzureOpenAI(
    azure_endpoint=api_base,
    api_key=api_key,
    api_version=api_version,)

In [0]:
system_message = "You are a helpful assistant."
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [0]:
def chat(message, history):
    if len(history)==0:
        messages = [{"role": "system", "content": system_message}] + [{"role": "user", "content": message}]
    else:
        updated_history = []
        for row in history:
            updated_history.append({'role':row["role"], "content":row["content"]})
        messages = [{"role": "system", "content": system_message}] + updated_history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=chat_model_id, messages=messages)
    return response.choices[0].message.content

In [0]:
chat("what is the for procedure code 58822 in MA state for Aetna healthplan?", [])

"I don't have access to specific insurance provider databases or the latest coding information; it's best to check directly with Aetna or a coding resource for accurate information regarding procedure code 58822 in Massachusetts."

In [0]:
df = spark.table("predictpx.default.healthplan_postalcode_temp")
def get_price(procedure_code):
    results = df.filter(col('code') == procedure_code).collect()
    results_dict = [row.asDict() for row in results]
    return results_dict[0:10]

In [0]:
price_function = {
    "name": "get_price",
    "description": "Get the price for procedure code, for example when a customer asks 'What is the price for procedure code 57410?'",
    "parameters": {
        "type": "object",
        "properties": {
            "procedure_code": {
                "type": "string",
                "description": "The procedure code to user get the price for.",
            },
        },
        "required": ["procedure_code"],
        "additionalProperties": False
    }
}

In [0]:
tools = [{"type": "function", "function": price_function}]

In [0]:
def chat(message, history):
    if len(history)==0:
        messages = [{"role": "system", "content": system_message}] + [{"role": "user", "content": message}]
    else:
        updated_history = []
        for row in history:
            updated_history.append({'role':row["role"], "content":row["content"]})
        messages = [{"role": "system", "content": system_message}] + updated_history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=chat_model_id, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, procedure_code = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=chat_model_id, messages=messages)
    
    return response.choices[0].message.content

In [0]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    procedure_code = arguments.get('procedure_code')
    price = get_price(procedure_code)
    response = {
        "role": "tool",
        "content": json.dumps({"procedure_code": procedure_code,"price": price}),
        "tool_call_id": tool_call.id
    }
    return response, procedure_code

In [0]:
output = chat("what is the for procedure code 58822 in MA state for Aetna healthplan?", [])
output

'The average negotiated price for procedure code 58822 in Massachusetts for Aetna health plan is $2,887.'